# Pipe Flow Heat Transport PDE Simulations with Pyomo

Solving the heat transport for incompressible fluid flow in a horizontal pipe:

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyomo as pyo

In [ ]:
plot_dir = "plots"
os.makedirs(plot_dir, exist_ok=True)

## Single Temperature Flow Model Simulation

**Solar collector section ($0 < x ≤ L$):**
$$\frac{\partial T}{\partial t} + v(t) \frac{\partial T}{\partial x} = \alpha \frac{\partial^2 T}{\partial x^2} + \frac{q_{eff}(t)}{\rho c_p} - \frac{4 h(T - T_{amb})}{D \rho c_p}$$

**Insulated pipe extension at outlet ($L < x ≤ L_{extended}$):**
$$\frac{\partial T}{\partial t} + v(t) \frac{\partial T}{\partial x} = \alpha \frac{\partial^2 T}{\partial x^2}$$

**Solar heat input:**
$$q_{eff}(t) = \frac{I(t) \cdot c \cdot \epsilon}{2}$$

Where:
- $T(x,t)$ : temperature at position x and time t [K]
- $v(t)$ : time-varying fluid velocity [m/s]
- $\alpha$ : thermal diffusivity [m²/s]
- $q_{eff}(t)$ : effective concentrated solar heat flux [W/m²]
- $I(t)$ : natural solar irradiance (DNI) [W/m²]
- $c$ : solar concentration factor (mirror width / absorber width) [-]
- $\epsilon$ : optical efficiency (mirror/alignment losses) [-]
- $c_p$ : specific heat capacity [J/kg·K]
- $h$ : convective heat transfer coefficient [W/m²·K]
- $D$ : pipe inner diameter [m]
- $T_{amb}$ : ambient temperature [K]

Note: The factor of 2 in the denominator accounts for the parabolic mirrors directing sunlight onto only 180° of the pipe surface.

In [ ]:
from solar_collector.solar_collector_dae_pyo import (
    ZERO_C,
    PIPE_DIAMETER,
    COLLECTOR_LENGTH,
    THERMAL_DIFFUSIVITY,
    create_collector_model,
    add_pde_constraints,
    solve_model,
    plot_results,
    print_temp_profiles,
)

model_name = "oil_temp"

In [ ]:
# Create and solve the model
print("Creating pipe flow heat transport model...")
model = create_collector_model(t_final=60.0 * 5)  # Simulate for 5 minutes

print("Adding PDE constraints...")
T_initial = ZERO_C + 270.0
model = add_pde_constraints(model, T_initial=T_initial)

print("Solving the discretized PDE...")
results = solve_model(model, n_x=110, n_t=50, tol=1e-6)

print(f"Solver status: {results.solver.status}")
print(f"Termination condition: {results.solver.termination_condition}")

if results.solver.termination_condition in ["optimal", "locallyOptimal"]:
    if results.solver.termination_condition == "locallyOptimal":
        print("WARNING: Solution found is only locally optimal.")
    print("Plotting results...")
    t_eval = ([0.0, 60.0, 120.0, 180.0, 240.0, 300.0],)
    x_eval = [0.0, 20.0, 30.0, 60.0, 80.0, 100.0, 110.0]
    fig1, fig2 = plot_results(model, t_eval=t_eval, x_eval=x_eval)
    fig1.tight_layout()
    filename = f"model_{model_name}_tsplots.png"
    fig1.savefig(os.path.join(plot_dir, filename), dpi=150)
    fig2.tight_layout()
    filename = f"model_{model_name}_oil_temp_field.png"
    fig2.savefig(os.path.join(plot_dir, filename), dpi=150)
    plt.show()
    print_temp_profiles(model, t_eval=t_eval, x_eval=x_eval)
else:
    print("Solution not optimal. Check model formulation.")
    print(f"Solver message: {results.solver.message}")

### Inspect Solution

In [ ]:
T = pd.Series(model.T.extract_values())
T.index.names = ["t", "x"]
T = T.unstack()
T.shape

## Empirical Heat Transfer Coefficient Formulas

In [ ]:
from solar_collector.solar_collector_dae_pyo_two_temp import (
    PIPE_DIAMETER_INT,
    FLUID_DENSITY,
    FLUID_DYNAMIC_VISCOSITY,
    FLUID_THERMAL_CONDUCTIVITY,
    FLUID_SPECIFIC_HEAT,
)
from solar_collector.heat_transfer import (
    calculate_heat_transfer_coefficient_nusselt,
    calculate_heat_transfer_coefficient_turbulent,
)

In [ ]:
pipe_diameter_int = PIPE_DIAMETER_INT
fluid_density = FLUID_DENSITY
fluid_viscosity = FLUID_DYNAMIC_VISCOSITY
fluid_thermal_conductivity = FLUID_THERMAL_CONDUCTIVITY
fluid_specific_heat = FLUID_SPECIFIC_HEAT

velocity = np.logspace(np.log10(0.02), np.log10(1.0), 101)

h_lam = calculate_heat_transfer_coefficient_nusselt(
    pipe_diameter_int, fluid_thermal_conductivity, Nu=4.36
)

h_turb, Re, Pr, Nu = calculate_heat_transfer_coefficient_turbulent(
    velocity,
    pipe_diameter_int,
    fluid_density,
    fluid_viscosity,
    fluid_thermal_conductivity,
    fluid_specific_heat,
)

fig, ax = plt.subplots(figsize=(7, 3))

ax.semilogx(
    velocity,
    h_turb,
    color="tab:blue",
    label="$h$ turbulent ($Nu = 0.023 * Re^{0.8} * Pr^{0.4}$)",
)
x_ticks = [0.02, 0.03, 0.05, 0.1, 0.2, 0.3, 0.5, 1.0]
ax.set_xticks(x_ticks)
ax.set_xticklabels(x_ticks)
ax.set_ylim([0, None])

ax.axhline(
    h_lam, color="tab:blue", linestyle="--", label="$h$ laminar ($Nu = 4.36$)"
)
ax2 = ax.twinx()
ax2.plot(velocity, Re, color="tab:orange", label="Reynold's number (Re)")
ax2.set_ylim([0, None])

v_min_turb = velocity[np.argmin((Re - 4000) ** 2)]
v_max_lam = velocity[np.argmin((Re - 2300) ** 2)]
xlim = ax2.get_xlim()
ylim = ax2.get_ylim()
y = np.linspace(ylim[0], ylim[1], 2)
ax2.fill_betweenx(
    y,
    np.full_like(y, xlim[0]),
    x2=np.full_like(y, v_max_lam),
    color="tab:green",
    alpha=0.1,
    label="laminar regime",
)
ax2.fill_betweenx(
    y,
    np.full_like(y, v_min_turb),
    x2=np.full_like(y, xlim[1]),
    color="tab:red",
    alpha=0.1,
    label="turbulent regime",
)

ax.set_xlabel("velocity [m/s]")
ax.set_ylabel("$h$ [W/m²·K]")
ax.grid()
ax.legend(loc="upper left")
ax.set_title("Dittus-Boelter Correlation for Thermal Oil")
ax2.set_ylabel("Re")
ax2.legend(loc="lower right")
plt.tight_layout()
plt.show()

## Fluid and Pipe Wall Temperature Model Simulation

**Two-Temperature Model with Fluid and Pipe Wall:**

This model considers separate temperatures for the fluid ($T_f$) and pipe wall ($T_p$) with heat transfer between them.

**Fluid temperature equation:**
$$\rho_f c_{p,f} \frac{\partial T_f}{\partial t} + \rho_f c_{p,f} v(t,x) \frac{\partial T_f}{\partial x} = \rho_f c_{p,f} D_{ax} \frac{\partial^2 T_f}{\partial x^2} + \frac{4 h_{int}(T_p - T_f)}{D}$$

**Pipe wall temperature equation:**

*Solar collector section ($0 < x ≤ L$):*
$$\rho_p c_{p,p} \frac{\partial T_p}{\partial t} = \rho_p c_{p,p} \alpha_p \frac{\partial^2 T_p}{\partial x^2} + \frac{4 q_{eff}(t) (D + 2d)}{(D + 2d)^2 - D^2} - \frac{4 h_{int}(T_p - T_f)}{D} - \frac{4 h_{ext} (D + 2d) (T_p - T_{amb})}{(D + 2d)^2 - D^2}$$

*Insulated pipe extension ($L < x ≤ L_{extended}$):*
$$\rho_p c_{p,p} \frac{\partial T_p}{\partial t} = \rho_p c_{p,p} \alpha_p \frac{\partial^2 T_p}{\partial x^2} - \frac{4 h_{int}(T_p - T_f)}{D} - \frac{4 h_{ext} (D + 2d) (T_p - T_{amb})}{(D + 2d)^2 - D^2}$$

**Velocity from mass flow rate (mass conservation):**
$$v(t,x) = \frac{\dot{m}(t)}{\rho_f(t,x) \cdot A}$$

**Solar heat input:**
$$q_{eff}(t) = \frac{I(t) \cdot c \cdot \epsilon}{2}$$

Where:
- $T_f(x,t)$ : fluid temperature [K]
- $T_p(x,t)$ : pipe wall temperature [K]
- $\dot{m}(t)$ : mass flow rate [kg/s]
- $v(t,x)$ : fluid velocity (varies with position when density is temperature-dependent) [m/s]
- $A$ : pipe cross-sectional area ($\pi D^2 / 4$) [m²]
- $\rho_f, \rho_p$ : fluid and pipe wall densities [kg/m³]
- $c_{p,f}, c_{p,p}$ : specific heat capacities [J/kg·K]
- $D_{ax}$ : axial dispersion coefficient for turbulent flow [m²/s]
- $\alpha_p = k_p/(\rho_p c_{p,p})$ : pipe wall thermal diffusivity [m²/s]
- $h_{int}$ : internal heat transfer coefficient (wall to fluid) [W/m²·K]
- $h_{ext}$ : external heat transfer coefficient (wall to ambient) [W/m²·K]
- $D$ : pipe inner diameter [m]
- $d$ : pipe wall thickness [m]
- $q_{eff}(t)$ : effective concentrated solar heat flux [W/m²]
- $I(t)$ : natural solar irradiance (DNI) [W/m²]
- $c$ : solar concentration factor (mirror width / absorber width) [-]
- $\epsilon$ : optical efficiency (mirror/alignment losses) [-]

Note: The factor of 2 in the denominator accounts for the parabolic mirrors directing sunlight onto only 180° of the pipe surface.

Boundary Conditions:

**At x = 0 (inlet)**

|     | T_f (fluid) | T_p (pipe wall) |
|-----|-------------|-----------------|
| PDE | Skipped (`if x == 0: Skip`) | Skipped (`if x == 0: Skip`) |
| BC  | Dirichlet: `T_f[t, 0] = T_inlet[t]` | Zero-gradient: `T_p[t, 0] = T_p[t, dx]` |
| IC  | Skipped at x=0 (governed by inlet BC) | Applied at x=0 |

**At x = L_ext (outlet)**

|     | T_f (fluid) | T_p (pipe wall) |
|-----|-------------|-----------------|
| PDE | Not skipped — still applied at x=L_ext | Not skipped — still applied at x=L_ext |
| BC  | Zero-gradient: `T_f[t, x_out] = T_f[t, x_out-1]` | Zero-gradient: `T_p[t, x_out] = T_p[t, x_out-1]` |

**Upwind Artificial Diffusion Stabilization**

The fluid temperature PDE is a convection-diffusion equation in which the advection term (transport by flow velocity $v$) typically dominates over the diffusion term (axial dispersion $D_{ax}$). For typical values — $v \approx 0.2$ m/s, $\Delta x \approx 1$ m, $D_{ax} = 10^{-4}$ m²/s — the cell Péclet number is:

$$Pe_{cell} = \frac{v \cdot \Delta x}{D_{ax}} \approx \frac{0.2 \times 1}{10^{-4}} = 2000$$

Central finite differences are only stable when $Pe_{cell} \leq 2$. At $Pe_{cell} = 2000$, the discretized system would produce unphysical spatial oscillations or diverge entirely.

The standard cure is to switch from central differences to first-order upwind differences for the advection term. This is mathematically equivalent to keeping central differences and adding an artificial diffusion of $v \cdot \Delta x / 2$ to the real diffusion coefficient. The code expresses this as:

```
D_eff = m.D_ax + m.v[t, x] * m.dx / 2
```

The cost: accuracy drops from second-order to first-order in $\Delta x$, but stability is guaranteed. As the grid is refined, $v \cdot \Delta x / 2 \to 0$ and $D_{eff} \to D_{ax}$.

Mathematical derivation

The physical PDE (fluid energy balance):
$$\rho c_p \frac{\partial T_f}{\partial t} + \rho c_p v \frac{\partial T_f}{\partial x} = \rho c_p D_{ax} \frac{\partial^2 T_f}{\partial x^2} +
q_{fluid}$$

With central differences for the first spatial derivative:
$$\frac{\partial T_f}{\partial x}\bigg|i \approx \frac{T{i+1} - T_{i-1}}{2\Delta x}$$

With first-order upwind (for $v > 0$):
$$\frac{\partial T_f}{\partial x}\bigg|i \approx \frac{T_i - T{i-1}}{\Delta x}$$

Rewrite the upwind approximation in terms of central differences:
$$\frac{T_i - T_{i-1}}{\Delta x} = \underbrace{\frac{T_{i+1} - T_{i-1}}{2\Delta x}}{\text{central}} - \underbrace{\frac{T{i+1} - 2T_i + T_{i-1}}{2\Delta
x}}_{\approx \frac{\Delta x}{2}\frac{\partial^2 T_f}{\partial x^2}}$$

Substituting into the PDE (multiply by $\rho c_p v$):
$$\rho c_p v \cdot \text{upwind} = \rho c_p v \cdot \text{central} - \rho c_p \cdot \underbrace{\frac{v \Delta x}{2}}_{\text{artificial diffusion}}
\cdot \frac{\partial^2 T_f}{\partial x^2}$$

Rearranging (moving artificial diffusion to the RHS):
$$\rho c_p \frac{\partial T_f}{\partial t} + \rho c_p v \frac{\partial T_f}{\partial x}\bigg|{\text{central}} = \rho c_p \underbrace{\left(D{ax} +
\frac{v \Delta x}{2}\right)}{D{eff}} \frac{\partial^2 T_f}{\partial x^2} + q_{fluid}$$

So D_eff = D_ax + v*dx/2 with central differences is algebraically identical to pure upwind differencing. The effective cell Péclet number becomes:
$$Pe_{eff} = \frac{v \Delta x}{D_{eff}} = \frac{v \Delta x}{D_{ax} + v\Delta x/2} \leq 2 \quad \text{always}$$

In [ ]:
from solar_collector.fluid_properties import SYLTHERM800
from solar_collector.solar_collector_dae_pyo_two_temp import (
    ZERO_C,
    create_collector_model,
    run_simulation,
    plot_results,
    print_temp_profiles,
)

model_name = "oil_wall_temp"

In [ ]:
# Create and solve the model
print("Creating pipe flow heat transport model...")

# Create fluid properties object
fluid_props = SYLTHERM800()

# Create model with constant fluid properties (evaluated at T_ref)
model = create_collector_model(
    fluid_props,
    t_final=60.0 * 5,  # Simulate for 5 minutes
    constant_density=True,
    constant_viscosity=True,
    constant_thermal_conductivity=True,
    constant_specific_heat=True,
    constant_heat_transfer_coeff=True,
)

print("Running simulation...")
T_f_initial = ZERO_C + 270.0
T_p_initial = ZERO_C + 210.0
results = run_simulation(
    model,
    T_f_initial=T_f_initial,
    T_p_initial=T_p_initial,
    n_x=110,
    n_t=50,
    tol=1e-6,
)

print(f"Solver status: {results.solver.status}")
print(f"Termination condition: {results.solver.termination_condition}")

if results.solver.termination_condition in ["optimal", "locallyOptimal"]:
    if results.solver.termination_condition == "locallyOptimal":
        print("WARNING: Solution found is only locally optimal.")
    print("Plotting results...")
    t_eval = ([0.0, 60.0, 120.0, 180.0, 240.0, 300.0],)
    x_eval = [0.0, 20.0, 30.0, 60.0, 80.0, 100.0, 110.0]
    fig1, fig2, fig3 = plot_results(model, t_eval=t_eval, x_eval=x_eval)
    fig1.tight_layout()
    filename = f"model_{model_name}_tsplots.png"
    fig1.savefig(os.path.join(plot_dir, filename), dpi=150)
    fig2.tight_layout()
    filename = f"model_{model_name}_oil_temp_field.png"
    fig2.savefig(os.path.join(plot_dir, filename), dpi=150)
    fig3.tight_layout()
    filename = f"model_{model_name}_wall_temp_field.png"
    fig3.savefig(os.path.join(plot_dir, filename), dpi=150)
    plt.show()
    print_temp_profiles(model, t_eval=t_eval, x_eval=x_eval)
else:
    print("Solution not optimal. Check model formulation.")
    print(f"Solver message: {results.solver.message}")